In [2]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load intents data from JSON file
with open('Intents.json') as file:

    intents_data = json.load(file)

# Extract data from intents
patterns = []
responses = []
tags = []
for intent in intents_data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern.lower())  # Convert to lowercase for consistency
        responses.append(intent['responses'][0])  # Assuming only one response per pattern
        tags.append(intent['tag'])

# Tokenize input text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patterns)

vocab_size = len(tokenizer.word_index) + 1
max_len = max([len(seq) for seq in tokenizer.texts_to_sequences(patterns)])

# Convert text data into sequences
pattern_sequences = tokenizer.texts_to_sequences(patterns)

# Pad sequences to make them of equal length
pattern_pad = pad_sequences(pattern_sequences, maxlen=max_len, padding='post')

# Convert tags to categorical labels
tag_labels = np.array(tags)
label_indices = {tag: i for i, tag in enumerate(set(tags))}
tag_labels = [label_indices[tag] for tag in tag_labels]

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len),
    LSTM(64),
    Dense(len(label_indices), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(pattern_pad, np.array(tag_labels), epochs=100, verbose=2)

# Function to generate a response
# Function to generate a response
def generate_response(user_input):
    user_input_seq = tokenizer.texts_to_sequences([user_input.lower()])
    user_input_pad = pad_sequences(user_input_seq, maxlen=max_len, padding='post')
    predicted_probs = model.predict(user_input_pad)[0]
    predicted_tag = np.argmax(predicted_probs)
    predicted_tag_name = [tag for tag, index in label_indices.items() if index == predicted_tag][0]
    for intent in intents_data['intents']:
        if intent['tag'] == predicted_tag_name:
            return np.random.choice(intent['responses'])
            


Epoch 1/100
2/2 - 1s - loss: 2.9992 - accuracy: 0.0500 - 758ms/epoch - 379ms/step
Epoch 2/100
2/2 - 0s - loss: 2.9954 - accuracy: 0.0500 - 8ms/epoch - 4ms/step
Epoch 3/100
2/2 - 0s - loss: 2.9924 - accuracy: 0.0500 - 9ms/epoch - 5ms/step
Epoch 4/100
2/2 - 0s - loss: 2.9899 - accuracy: 0.0667 - 7ms/epoch - 3ms/step
Epoch 5/100
2/2 - 0s - loss: 2.9870 - accuracy: 0.0667 - 9ms/epoch - 5ms/step
Epoch 6/100
2/2 - 0s - loss: 2.9847 - accuracy: 0.0667 - 7ms/epoch - 3ms/step
Epoch 7/100
2/2 - 0s - loss: 2.9816 - accuracy: 0.1000 - 10ms/epoch - 5ms/step
Epoch 8/100
2/2 - 0s - loss: 2.9782 - accuracy: 0.1000 - 10ms/epoch - 5ms/step
Epoch 9/100
2/2 - 0s - loss: 2.9745 - accuracy: 0.1667 - 9ms/epoch - 4ms/step
Epoch 10/100
2/2 - 0s - loss: 2.9708 - accuracy: 0.2000 - 10ms/epoch - 5ms/step
Epoch 11/100
2/2 - 0s - loss: 2.9656 - accuracy: 0.1833 - 8ms/epoch - 4ms/step
Epoch 12/100
2/2 - 0s - loss: 2.9596 - accuracy: 0.1833 - 10ms/epoch - 5ms/step
Epoch 13/100
2/2 - 0s - loss: 2.9529 - accuracy: 0.20

In [3]:
# Test the chatbot

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input)
    print("Bot:", response)

1/1 [==============================] - 0s 199ms/step
Bot: You're welcome! If you need anything else, feel free to ask.
1/1 [==============================] - 0s 16ms/step
Bot: You're welcome! If you need anything else, feel free to ask.
1/1 [==============================] - 0s 16ms/step
Bot: You're welcome! If you need anything else, feel free to ask.
1/1 [==============================] - 0s 16ms/step
Bot: You're welcome! If you need anything else, feel free to ask.
1/1 [==============================] - 0s 19ms/step
Bot: Hello! Welcome to our restaurant. How can I assist you today?
1/1 [==============================] - 0s 16ms/step
Bot: Of course! What would you like to order?
